In [1]:
import os
import math
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, cdist
from mpl_toolkits.mplot3d import Axes3D
import soundfile as sf
from prettytable import PrettyTable
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.signal import hilbert, butter, filtfilt, medfilt, lfilter, wiener, convolve


In [2]:
fruit_types = ['pera', 'banana', 'manzana', 'naranja']
audios = {fruit: [] for fruit in fruit_types}
root_dir = '../../dataset'



In [3]:
for dirname, _, filenames in os.walk(root_dir):
    fruit_type = os.path.basename(dirname)
    if fruit_type in fruit_types:
        audios[fruit_type].extend([os.path.join(dirname, filename) for filename in filenames if filename.endswith('.wav')])


In [4]:
processed = {fruit: [] for fruit in fruit_types}

for dirname, _, filenames in os.walk(root_dir):
    path = os.path.basename(dirname)
    if path == 'processed':
        fruit_type = os.path.basename(os.path.dirname(dirname))
        if fruit_type in fruit_types:
            processed[fruit_type].extend([os.path.join(dirname, filename) for filename in filenames if filename.endswith('.wav')])



In [5]:
FRAME_SIZE = 1024 # In the documentation says it's convenient for speech.C
HOP_SIZE   = int(FRAME_SIZE/2)


In [6]:
def load_audio(audiofile):
    test_audio, sr = librosa.load(audiofile, sr = None)
    duration = librosa.get_duration(filename=audiofile, sr=sr)
    return test_audio, sr, duration


In [7]:
def time_vector(signal, duration):
    return np.linspace(0, duration, len(signal))



In [8]:
def rms(signal):
    return librosa.feature.rms(y=signal, frame_length = FRAME_SIZE, hop_length = HOP_SIZE)



In [9]:
def normalize(signal):
    peak = np.max(signal)
    signal/=peak
    return signal



In [10]:
def low_pass_filter(signal, sr, cutoff_frequency = 5000):
    nyquist = 0.5 * sr
    cutoff = cutoff_frequency / nyquist
    b, a = butter(N=6, Wn=cutoff, btype='low', analog=False, output='ba')
    filtered = lfilter(b, a, signal)
    return filtered


In [ ]:
def band_pass_filter(signal, sr, low_cutoff, high_cutoff):
    b, a = butter(N=3, Wn = [low_cutoff, high_cutoff], btype='band', fs=sr)
    return lfilter(b, a, signal)
def wiener_filter(signal, noise = 0.9):
    filtered = wiener(signal, noise = noise)
    return filtered
def envelope(signal):
    analytic_signal = hilbert(signal)
    return np.abs(analytic_signal)

def smooth_envelope(signal, sr, cutoff_frequency=50.0):
    return low_pass_filter(envelope(signal), sr, cutoff_frequency)



In [11]:
#2d
def plot_features2d(features):
    fig = plt.figure()
    colors = dict(zip(fruit_types,['green','yellow','red','orange']))
    

    for fruit, points in features.items():
        plt.scatter(points[:, 0], points[:, 1], c = colors[fruit], label=fruit)

    plt.xlabel('Eje X')
    plt.ylabel('Eje Y')
    plt.show()


In [ ]:

cutoff = 2000
features = dict.fromkeys(fruit_types)
split_frequency = 2000

# Extracción de características
for fruit, group in processed.items():
    features[fruit] = None
    for audio in group:
        signal, sr, duration = load_audio(audio)
        audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

        #spec = librosa.stft(signal, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
        #BER  = band_energy_ratio(spec, split_frequency, sr)
        #BER /= np.max(BER)
        #feat = np.mean(BER)

        #centroidal = librosa.feature.spectral_centroid(y=signal, sr=sr, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)[0]
        #centroidal /= np.max(centroidal)
        #centroidal = np.mean(centroidal)
        #smoothed = rms(signal)
        #smoothed = smoothed.reshape(-1,)
        #rms_smoothed = np.mean(smoothed)/np.max(smoothed)
        #filtered = low_pass_filter(signal, sr, cutoff)
        #feat = librosa.feature.zero_crossing_rate(signal, frame_length=FRAME_SIZE, hop_length=HOP_SIZE)[0]
        #max = np.max(zcr)
        #flux = spectral_flux(filtered)
        #max = np.max(flux)
        #flux /= max
        #flux = np.mean(flux)
        #roll_off = librosa.feature.spectral_rolloff(y=signal, sr=sr, n_fft=FRAME_SIZE, hop_length=HOP_SIZE, roll_percent=0.85)[0]
        #max = np.max(roll_off)
        #roll_off /= max
        #roll_off = np.mean(roll_off)
        #smoothed /= np.max(smoothed)
        #N = 1
        #feat = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 5, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
        #feat = librosa.feature.delta(feat, order = 3)

        #frames = range(len(feat))
        #t = librosa.frames_to_time(frames, sr=sr, n_fft = FRAME_SIZE, hop_length=HOP_SIZE)

        #absolute = np.abs(feat)
        #feat /= np.max(absolute)
        #absolute /= np.max(absolute)
        #feat = np.abs(feat)
        #row = feat[N,:]
        #row = np.abs(row)

        #momentum = np.dot(t, absolute)
        #momentum/=np.sum(absolute)
        #means = np.mean(feat)

        feature = np.array([[audio_rms, feat]])
        if features[fruit] is not None:
            features[fruit] = np.vstack([features[fruit], feature])
        else:
            features[fruit] = feature
# Ploteo
plot_features2d(features)




**MFCC**

- Media

In [ ]:
cutoff = 6500
cuton = 50
features = dict.fromkeys(fruit_types)
split_frequency = 200


# Extracción de características
for fruit, group in processed.items():
    features[fruit] = None
    for audio in group:
        signal, sr, duration = load_audio(audio)
        audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

        mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 30, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
        mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)
        feat = np.mean(mfccs, axis = 1)
        componente = 26
        feature = np.array([[audio_rms, feat[componente]]])
        if features[fruit] is not None:
            features[fruit] = np.vstack([features[fruit], feature])
        else:
            features[fruit] = feature
# Ploteo
plot_features2d(features)


- RMS. Mejor la media que el RMS

In [ ]:
cutoff = 6500
cuton = 50
features = dict.fromkeys(fruit_types)
split_frequency = 200


# Extracción de características
componnte = 0
while componente < 30:
    for fruit, group in processed.items():
        features[fruit] = None
        for audio in group:
            signal, sr, duration = load_audio(audio)
            audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

            mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 30, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)
            feat = np.sqrt(np.mean(mfccs**2, axis=1))


            feature = np.array([[audio_rms, feat[componente]]])
            if features[fruit] is not None:
                features[fruit] = np.vstack([features[fruit], feature])
            else:
                features[fruit] = feature
    # Ploteo
    plot_features2d(features)
    input()
    componente += 1


- Máximo. En la componente 3 se separan las manzanas de las peras

In [ ]:
cutoff = 6500
cuton = 50
features = dict.fromkeys(fruit_types)
split_frequency = 200


# Extracción de características
componente = 0
while componente < 30:
    for fruit, group in processed.items():
        features[fruit] = None
        for audio in group:
            signal, sr, duration = load_audio(audio)
            audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

            mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 30, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            feat = np.max(mfccs, axis = 1)
            mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)

            feature = np.array([[audio_rms, feat[componente]]])
            if features[fruit] is not None:
                features[fruit] = np.vstack([features[fruit], feature])
            else:
                features[fruit] = feature
    # Ploteo
    plot_features2d(features)
    input()
    componente += 1


- Mínimo

In [ ]:
cutoff = 6500
cuton = 50
features = dict.fromkeys(fruit_types)
split_frequency = 200


# Extracción de características
componente = 0
while componente < 30:
    for fruit, group in processed.items():
        features[fruit] = None
        for audio in group:
            signal, sr, duration = load_audio(audio)
            audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

            mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 30, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            feat = np.min(mfccs, axis = 1)
            mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)

            feature = np.array([[audio_rms, feat[componente]]])
            if features[fruit] is not None:
                features[fruit] = np.vstack([features[fruit], feature])
            else:
                features[fruit] = feature
    # Ploteo
    plot_features2d(features)
    input()
    componente += 1


- Varianza. La componente 1 mas o menos separa las bananas de las naranjas

In [ ]:
cutoff = 6500
cuton = 50
features = dict.fromkeys(fruit_types)
split_frequency = 200


# Extracción de características
componente = 0
while componente < 30:
    for fruit, group in processed.items():
        features[fruit] = None
        for audio in group:
            signal, sr, duration = load_audio(audio)
            audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

            mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 30, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)
            feat = np.var(mfccs, axis = 1)

            feature = np.array([[audio_rms, feat[componente]]])
            if features[fruit] is not None:
                features[fruit] = np.vstack([features[fruit], feature])
            else:
                features[fruit] = feature
    # Ploteo
    plot_features2d(features)
    input()
    componente += 1


- Desviación estándar. Lo mismo que la varianza respecto de la componente 1

In [52]:
cutoff = 6500
cuton = 50
features = dict.fromkeys(fruit_types)
split_frequency = 200


# Extracción de características
componente = 0
while componente < 30:
    for fruit, group in processed.items():
        features[fruit] = None
        for audio in group:
            signal, sr, duration = load_audio(audio)
            audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

            mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 30, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)
            feat = np.std(mfccs, axis = 1)/np.mean(mfccs, axis = 1)

            feature = np.array([[audio_rms, feat[componente]]])
            if features[fruit] is not None:
                features[fruit] = np.vstack([features[fruit], feature])
            else:
                features[fruit] = feature
    # Ploteo
    plot_features2d(features)
    input()
    componente += 1


KeyboardInterrupt: Interrupted by user

- Con el momento sin la suma

In [ ]:
cutoff = 6500
cuton = 50
features = dict.fromkeys(fruit_types)
split_frequency = 200


# Extracción de características
componente = 0
while componente < 30:
    for fruit, group in processed.items():
        features[fruit] = None
        for audio in group:
            signal, sr, duration = load_audio(audio)
            audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

            mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 30, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)
            frames = range(mfccs.shape[1])

            t = librosa.frames_to_time(frames, sr=sr, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            feat = np.dot(mfccs, t)

            feature = np.array([[audio_rms, feat[componente]]])
            if features[fruit] is not None:
                features[fruit] = np.vstack([features[fruit], feature])
            else:
                features[fruit] = feature
    # Ploteo
    plot_features2d(features)
    input()
    componente += 1


- Con el momento con suma. Buena separación de las peras respecto de las demas con la componente 0

In [ ]:
cutoff = 6500
cuton = 50
features = dict.fromkeys(fruit_types)
split_frequency = 200


# Extracción de características
componente = 0
while componente < 30:
    for fruit, group in processed.items():
        features[fruit] = None
        for audio in group:
            signal, sr, duration = load_audio(audio)
            audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)

            mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = 30, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)
            frames = range(mfccs.shape[1])

            t = librosa.frames_to_time(frames, sr=sr, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
            feat = np.dot(np.abs(mfccs), t)/np.sum(np.abs(mfccs), axis = 1)

            feature = np.array([[audio_rms, feat[componente]]])
            if features[fruit] is not None:
                features[fruit] = np.vstack([features[fruit], feature])
            else:
                features[fruit] = feature
    # Ploteo
    plot_features2d(features)
    input()
    componente += 1
